Меняем среду выполнения на T4 GPU

In [ ]:
!pip3 install onnx vosk-tts

In [ ]:
!git clone https://github.com/alphacep/vosk-tts

Создаём папку monotonic_align внутри monotonic_align, потому что в vosk-tts/training/monotonic_align/setup.py указан относительный путь monotonic_align. Запускаем setup.py

In [ ]:
%cd vosk-tts/training
%cd monotonic_align
%mkdir monotonic_align
!python setup.py build_ext --inplace

In [ ]:
%cd ..

Скачиваем обученную модель

In [ ]:
!git clone https://huggingface.co/alphacep/vosk-tts-ru-multi pretrained

Cloning into 'pretrained'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 19 (delta 2), reused 0 (delta 0), pack-reused 4
Unpacking objects: 100% (19/19), 1.68 MiB | 11.03 MiB/s, done.
Filtering content: 100% (3/3), 1010.85 MiB | 82.39 MiB/s, done.


In [ ]:
!wget https://alphacephei.com/vosk/models/vosk-model-tts-ru-0.4-multi.zip
!unzip vosk-model-tts-ru-0.4-multi.zip
%mkdir -p db
%cp vosk-model-tts-ru-0.4-multi/dictionary db/dictionary

Скачиваем существующий пакет голосовой адаптации

In [ ]:
!wget https://alphacephei.com/tts/db-finetune.zip
!unzip db-finetune.zip

Мы хотим обучать модель на других примерах, поэтому удаляем все аудиозаписи

In [ ]:
import os, shutil


folder = '/content/vosk-tts/training/db-finetune/wavs'

for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

Скачиваем необходимые библиотеки: pydub - для того, чтобы привести наши аудиозаписи к необходимой частоте дискретизации 22050 герц, wget и PyDrive - чтобы скачать наши данные с гугл диска.

Важно! Записи в датасете должны быть не больше 10-15 секунд длиной, иначе неконтролируемо вознакают ошибки

In [ ]:
!pip install pydub
!pip install wget
!pip install -U -q PyDrive

Скачиваем датасет

In [ ]:
import os
import wget
import wave
import contextlib
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#FOLDER_ID берём из ссылки формата https://drive.google.com/drive/folders/YOUR_FOLDER_ID

FOLDER_ID = '1rBujDC7qd-J8cEoDoEkQLRd9UQiSQibs' # Датасет с Павлом Волей
file_list = drive.ListFile({'q': f"'{FOLDER_ID}' in parents"}).GetList()
PATH_TO_FINETUNE = '/content/vosk-tts/training/db-finetune/'
PATH_TO_WAVS = '/content/vosk-tts/training/db-finetune/wavs/'

for i, f in enumerate(file_list):
    if f['mimeType'] == 'audio/wav':
        response = wget.download(f['webContentLink'], os.path.join(PATH_TO_WAVS, f['title']))
        print(response)
    elif f['mimeType'] == 'text/csv':
        response = wget.download(f['webContentLink'], os.path.join(PATH_TO_FINETUNE, 'metadata-unformatted.csv'))
        print(response)

Изменяем частоту дискретизации записей

In [ ]:
import shutil
import os
from tqdm import tqdm
from pydub import AudioSegment as am

def rerate(filepath):
    sound = am.from_file(filepath, format='wav')
    sound = sound.set_frame_rate(22050)
    sound.export(filepath, format='wav')

for filename in tqdm(os.listdir(PATH_TO_WAVS)):
    rerate(os.path.join(PATH_TO_WAVS, filename))

Приводим метадату к нужному формату

In [ ]:
reference = open(os.path.join(PATH_TO_FINETUNE, 'metadata-unformatted.csv'), 'r')

with open(os.path.join(PATH_TO_FINETUNE, 'metadata.csv'), 'w') as f:
    for i, line in enumerate(tqdm(reference)):
        pos = line.find(',')
        filename = line[:pos]
        text = line[pos+1:].strip()
        if (text.startswith('"')):
            text = text[1:-1]
        f.write('db-finetune/wavs/' + filename + '|0|' + text + '|_\n')

Генерируем выборки для обучения и тестирования

In [ ]:
!python3 /content/vosk-tts/training/db-finetune/traintest.py '/content/vosk-tts/training/db-finetune/metadata.csv' '10'
# '10' означает, что каждый 10 элемент попадёт в тестирующую выборку

В файле train_finetune.py нужно заменить строчку 58 на **`run(rank=0, n_gpus=1, hps=hps)`** -  избавляемся от мультипроцессинга, т.к. аудио - тяжелый формат, и колаб не сможет работать с ним в формате мультипроцессинга

В db-finetune/config.json можно изменить количество эпох и другие параметры дообучения

Запускаем дообучение

In [ ]:
!python3 /content/vosk-tts/training/train_finetune.py

В файле stft.py в 206 строчке меняем
**`fft_window = pad_center(fft_window, filter_length)`**
на
**`fft_window = pad_center(fft_window, size=filter_length)`**


В файле onnx_export.py в 30 строчке меняем путь до последнего G_чекпоинта, он будет показан в выводе при завершении дообучения

Экспортированная модель будет лежать по такому пути:

In [ ]:
%pwd

Экспортируем модель

In [ ]:
!python3 onnx_export.py

Новую модель нужно переместить в папку vosk-model-ru-0.4-multi, удалив перед эти старую модель

Посмотрим на результат обучения

In [ ]:
from vosk_tts import Model, Synth
import IPython

PATH_TO_MODEL = "/content/vosk-tts/training/vosk-model-tts-ru-0.4-multi"
PATH_TO_OUTPUT = "/content/vosk-tts/training"


class VoiceGenerator:
    def __init__(self):
        self.model = Model(model_path=PATH_TO_MODEL)
    def generate(self, text):
        synth = Synth(self.model)
        path = os.path.join(PATH_TO_OUTPUT, "output.wav")
        synth.synth(text, path)
        return path


# Example of usage
vg = VoiceGenerator()

/content/vosk-tts/training/output.wav


In [ ]:
path_to_file = vg.generate("Я Павел Воля!")
print(path_to_file)
IPython.display.Audio(path_to_file)